In [47]:
import torch
import torchvision
import torchvision.transforms as transforms

In [48]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [182]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [131]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#criterion = nn.MSELoss() torch.argmax(outputs, dim = 1)
# criterion = nn.NLLLoss
# criterion = nn.KLDivLoss

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [183]:
# softmax = nn.LogSoftmax(dim=1)

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.303
[1,  4000] loss: 2.303
[1,  6000] loss: 2.303
[1,  8000] loss: 2.304
[1, 10000] loss: 2.304
[1, 12000] loss: 2.304
[2,  2000] loss: 2.304
[2,  4000] loss: 2.303
[2,  6000] loss: 2.304
[2,  8000] loss: 2.303
[2, 10000] loss: 2.304
[2, 12000] loss: 2.304
[3,  2000] loss: 2.304
[3,  4000] loss: 2.304
[3,  6000] loss: 2.303
[3,  8000] loss: 2.304
[3, 10000] loss: 2.304
[3, 12000] loss: 2.303
[4,  2000] loss: 2.304
[4,  4000] loss: 2.304
[4,  6000] loss: 2.304
[4,  8000] loss: 2.303
[4, 10000] loss: 2.304
[4, 12000] loss: 2.304
[5,  2000] loss: 2.304
[5,  4000] loss: 2.303
[5,  6000] loss: 2.303
[5,  8000] loss: 2.305
[5, 10000] loss: 2.304
[5, 12000] loss: 2.303
[6,  2000] loss: 2.304
[6,  4000] loss: 2.303
[6,  6000] loss: 2.304
[6,  8000] loss: 2.303
[6, 10000] loss: 2.304
[6, 12000] loss: 2.304
[7,  2000] loss: 2.304
[7,  4000] loss: 2.303
[7,  6000] loss: 2.304
[7,  8000] loss: 2.303
[7, 10000] loss: 2.304
[7, 12000] loss: 2.303
[8,  2000] loss: 2.303
[8,  4000] 

In [184]:
import torch
import torch.nn as nn

# size of input (N x C) is = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# every element in target should have 0 <= value < C
target = torch.tensor([1, 0, 4])

softmax = nn.LogSoftmax(dim=1)
nll_loss = nn.NLLLoss()
output = nll_loss(softmax(input), target)
output.backward()

print('input: ', input)
print('target: ', target)
print('output: ', output)

input:  tensor([[ 1.5398,  1.3991,  0.6246, -0.9694, -1.3747],
        [-0.0281,  0.9032,  1.6414,  0.0165,  0.9078],
        [ 2.3238, -0.2847,  0.4932, -0.2474,  0.0907]], requires_grad=True)
target:  tensor([1, 0, 4])
output:  tensor(2.0404, grad_fn=<NllLossBackward0>)


In [185]:
PATH = './cifar_net_q12_1.pth'
torch.save(net.state_dict(), PATH)

In [186]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 9 %


In [187]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 0.0 %
Accuracy for class: car   is 0.0 %
Accuracy for class: bird  is 0.1 %
Accuracy for class: cat   is 0.0 %
Accuracy for class: deer  is 0.0 %
Accuracy for class: dog   is 0.0 %
Accuracy for class: frog  is 85.1 %
Accuracy for class: horse is 0.0 %
Accuracy for class: ship  is 13.5 %
Accuracy for class: truck is 0.0 %
